In [1]:
import ipyvuetify as v

from traitlets import (Any, Bool, Dict, Int, Unicode, List)


import openalea.strawberry.application.misc as misc
import openalea.strawberry.application.app_import_data as p1
import openalea.strawberry.application.app_visualization as p2
import openalea.strawberry.application.app_plant_scale as p3
import openalea.strawberry.application.app_module_scale as p4
import openalea.strawberry.application.app_node_scale as p5

## Initialize global variable

In [2]:
misc.init_allmtg()

# Main Container

In [3]:
container_main = v.Container(
    _metadata={'mount_id': 'content-main'}, 
    children=[
        p1.p1_container_main
    ]
)
# container_main

# Navigation

In [5]:
# Navigation (menubar)
components_map = {
    'Import MTG': p1.p1_container_main,
    'Visualization': p2.p2_container_main,
    'Plant Scale': p3.p3_container_main,
    'Module Scale': p4.p4_container_main,
    'Node Scale': p5.p5_container_main,
}

py_items = [
        { 'title': 'Import MTG', 'icon': 'mdi-view-dashboard'},
        { 'title': 'Visualization', 'icon': 'mdi-image'},
        { 'title': 'Plant Scale', 'icon': 'mdi-help-box'},
        { 'title': 'Module Scale', 'icon': 'mdi-view-dashboard'},
        { 'title': 'Node Scale', 'icon': 'mdi-image'},
      ]
      
class MyNavigation(v.VuetifyTemplate):
    items = List(py_items).tag(sync=True)
    
    template = Unicode('''  
        <v-list
          dense
          nav
        >
          <v-list-item
            v-for="item in items"
            :key="item.title"
            link
            @click="item_click(item)"
          >
            <v-list-item-icon>
              <v-icon>{{ item.icon }}</v-icon>
            </v-list-item-icon>
  
            <v-list-item-content>
              <v-list-item-title>{{ item.title }}</v-list-item-title>
            </v-list-item-content>
          </v-list-item>
        </v-list>
    ''').tag(sync=True)
    
    def vue_item_click(self, data):
        component = components_map[data['title']]
        container_main.children = [component]

navigation = MyNavigation()

container_nav = v.Container(
    _metadata={'mount_id': 'content-nav'}, 
    children=[
        navigation
    ]
)
# container_nav

Container(children=[MyNavigation(events=['item_click'], items=[{'title': 'Import MTG', 'icon': 'mdi-view-dashb…